In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import sys
sys.path.insert(0, '/home/tomek/ib_tools/')
from indicators import get_signals

In [2]:
data = pd.read_pickle('freeze/freeze_df_NQZ9.pickle')

In [3]:
data.head(40)

,backfill,open,high,low,close,price,volume,ema_fast,atr,signal
date,,,,,,,,,,
2018-02-01 04:48:00,True,6980.0,6988.0,6964.0,6987.0,6978.559502,14920,6978.559502,24.000000,0.0
2018-02-01 11:34:00,True,6987.0,7011.0,6962.0,6973.0,6975.138629,14963,6976.834812,36.656250,0.0
2018-02-01 14:12:00,True,6972.0,6973.0,6947.0,6955.0,6957.368682,15001,6970.237659,33.014999,0.0
2018-02-01 14:44:00,True,6955.0,6955.0,6928.0,6929.0,6938.128569,15382,6962.008599,31.454395,0.0
2018-02-01 15:02:00,True,6929.0,6933.0,6904.0,6916.0,6915.561298,15111,6952.406932,30.938669,0.0
2018-02-01 15:27:00,True,6916.0,6918.0,6898.0,6913.0,6907.656372,15055,6944.634314,28.999739,0.0
2018-02-01 15:33:00,True,6913.0,6933.0,6910.0,6933.0,6922.999773,16067,6941.387005,28.077028,0.0
2018-02-01 15:39:00,True,6933.0,6946.0,6930.0,6941.0,6939.452989,15096,6941.130918,26.432049,0.0
2018-02-01 15:48:00,True,6941.0,6943.0,6923.0,6928.0,6932.019692,16758,6940.049768,25.643853,0.0


In [4]:
df = pd.DataFrame(data[['open', 'close']])
df['volume_weighted'] = (df.close + df.open)/2 * df.volume
weighted_price = df.volume_weighted.sum() / df.volume.sum()

AttributeError: 'DataFrame' object has no attribute 'volume'

In [ ]:
periods = [10, 20, 40, 80, ]

In [ ]:
min_max = pd.DataFrame({'close': data.price})

In [ ]:
min_max

In [ ]:
for p in periods:
    min_max[f'min_{p}'] = min_max.close.rolling(p).min()
    #min_max[f'max_{p}'] = min_max.close.rolling(p).max()

In [ ]:
min_max.plot(figsize=(18, 9), grid=True);

In [ ]:
data[data['signal'] != 0].loc['20190927':].head(50)

In [ ]:
data[data['signal'] != 0].tail(60)

In [ ]:
averages = pd.DataFrame({'price': data.price,
                       'ema_fast': data.price.ewm(span=100).mean(),
                       'ema_slow': data.price.ewm(span=160).mean()})

In [ ]:
averages.plot(figsize=(18,9), grid=True);